In [126]:
from manim import *
import math
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching ndw_bernoulli_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s ndw_bernoulli_Scene"

In [61]:
background_color = "#20455A"

In [348]:
class BernoulliTube(Mobject):
    def __init__(self, tube_center = np.array([-2, -1.5, 0]), tube_stroke_color = WHITE, v0 = 1, **kwargs):
        super().__init__(**kwargs)

        self.center = tube_center
        self.tube_height = 1.5
        self.tube_width = 5
        self.tube_line_width = 5
        self.tube_stroke_color = tube_stroke_color
        self.v0 = v0
        self.count = 0

        # coordinate system: x in [0, tube_width], y in [-1, 1]
        self.tube_ax = Axes(x_range = [0, self.tube_width, 1], y_range = [-1, 1, 1], x_length = self.tube_width, y_length = 2*self.tube_height, tips = False).set_color(BLACK).move_to(self.center)
        # self.add(self.tube_ax)

        # bernoulli tube
        erf_up = self.get_erf(1)
        erf_down = self.get_erf(-1)
        tube_tunnel_up = self.tube_ax.plot(erf_up, color = self.tube_stroke_color, stroke_width = self.tube_line_width, x_range = [-0.1, 5])
        tube_tunnel_down = self.tube_ax.plot(erf_down, color = self.tube_stroke_color, stroke_width = self.tube_line_width, x_range = [-0.1, 5])

        # bernoulli tube background
        polygon_corner_list = [
            *[self.tube_ax.c2p(x, erf_up(x), 0) for x in np.linspace(-0.1, 5, 100)],
            *[self.tube_ax.c2p(x, erf_down(x), 0) for x in np.linspace(5, -0.1, 100)]
        ]
        tube_background_polygon = Polygon(*polygon_corner_list, fill_color = WHITE, fill_opacity = 1, stroke_opacity = 0, stroke_width = self.tube_line_width)
        tube_polygon = Polygon(*polygon_corner_list, fill_color = BLUE, fill_opacity = 0.25, stroke_opacity = 0, stroke_width = self.tube_line_width)

        # counter threshold
        counter_line = DashedLine(start = self.center + self.tube_height/4*UP + self.tube_width/2*RIGHT, end = self.center - self.tube_height/4*UP + self.tube_width/2*RIGHT, color = self.tube_stroke_color)
        counter_line.z_index = 5
        self.add(tube_background_polygon, tube_polygon)
        self.add(tube_tunnel_up, tube_tunnel_down, counter_line)



    # returns error function for a given initial value y0
    def get_erf(self, y0):
        x_center = self.tube_width * 2/5
        y_center = y0 * 1.25 / 2
        def errorfunction(x):
            return -y0*(2-1.25)/2*math.erf(x-x_center) + y_center
        return errorfunction
    

    # draws the errorfunctions
    def draw_erf(self, y0_list):
        plot_group = VGroup()
        for y0 in y0_list:
            erf = self.get_erf(y0)
            erf_plot = self.tube_ax.plot(erf, color = BLUE, stroke_width = 1, x_range = [0, 4.9])
            plot_group.add(erf_plot)
        return plot_group


    # calculate the cross section depending on the x coordinate
    def __get_cross_section(self, x):
        erf = self.get_erf(1)
        return erf(x)**2 * PI
        

    # calculate the velocity depending on the x coordinate
    def get_velocity(self, x):
        A0 = self.__get_cross_section(0)
        Ax = self.__get_cross_section(x)
        return self.v0 * A0 / Ax
    

    # calculate the x coordinate for a given time
    def update_pos(self, pos, y0, dt = 0.01):
        x, y, _ = pos
        new_x = x + self.get_velocity(x) * dt
        # set back the x coordinate and increase counter
        if new_x > self.tube_width:
            new_x %= self.tube_width
            self.count += 1
        new_y = self.get_erf(y0)(new_x)
        return np.array([new_x, new_y, 0])
    

    # takes a position and returns the ax position
    def pos_to_ax(self, pos):
        return self.tube_ax.c2p(*pos)
    

    # gets a dot
    def init_dot(self, y0):
        dot = Dot(radius = 0.05, color = BLUE)
        dot.y0 = y0
        dot.position = np.array([0, y0, 0])
        dot.move_to(self.pos_to_ax(dot.position))
        return dot
            

    def get_counter(self, n = 0):
        counter_bg = Rectangle(height = 1, width = self.tube_width/5*2, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = 0.95).move_to(self.center + 1.5/5*self.tube_width*RIGHT + 3*UP)
        counter_number = Text(f"{n}", color = BLACK).move_to(self.center + 1.5/5*self.tube_width*RIGHT + 3*UP)
        counter_bg.z_index = -1
        return VGroup(counter_bg, counter_number)



class DoorHallway(Mobject):
    def __init__(self, hallway_center = np.array([2, -1.5, 0]), tube_stroke_color = WHITE, v0 = 1, **kwargs):
        super().__init__(**kwargs)

        self.center = hallway_center
        self.tube_height = 1.5
        self.tube_width = 5
        self.tube_line_width = 5
        self.tube_stroke_color = tube_stroke_color
        self.v0 = v0
        self.count = 0

        # define main drift tube coordinates
        self.up_left = self.center + self.tube_height*UP - self.tube_width/2*LEFT
        self.up_mid_left = self.center + self.tube_height*UP - self.tube_width/10*RIGHT
        self.up_mid_right = self.center + self.tube_height/4*UP - self.tube_width/10*RIGHT
        self.up_right = self.center + self.tube_height/4*UP - self.tube_width/2*RIGHT

        self.down_left = self.center + self.tube_height*DOWN - self.tube_width/2*LEFT
        self.down_mid_left = self.center + self.tube_height*DOWN - self.tube_width/10*RIGHT
        self.down_mid_right = self.center + self.tube_height/4*DOWN - self.tube_width/10*RIGHT
        self.down_right = self.center + self.tube_height/4*DOWN - self.tube_width/2*RIGHT

        # bernoulli tube main lines
        hallway_tunnel_up = Line(start = self.up_mid_right, end = self.up_right, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)
        hallway_tunnel_down = Line(start = self.down_mid_right, end = self.down_right, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)
        hallway_funnel_up = Line(start = self.up_mid_left, end = self.up_mid_right, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)
        hallway_funnel_down = Line(start = self.down_mid_left, end = self.down_mid_right, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)
        hallway_tunnel_up_left = Line(start = self.up_mid_left, end = self.up_left, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)
        hallway_tunnel_down_left = Line(start = self.down_mid_left, end = self.down_left, stroke_color = self.tube_stroke_color, stroke_width = self.tube_line_width)

        # bernoulli tube background
        polygon_corner_list = [
            self.up_left,
            self.up_mid_left,
            self.up_mid_right,
            self.up_right,
            self.down_right,
            self.down_mid_right,
            self.down_mid_left,
            self.down_left
        ]
        tube_background_polygon = Polygon(*polygon_corner_list, fill_color = WHITE, fill_opacity = 1, stroke_opacity = 0, stroke_width = self.tube_line_width)
        tube_polygon = Polygon(*polygon_corner_list, fill_color = GRAY, fill_opacity = 0.25, stroke_opacity = 0, stroke_width = self.tube_line_width)

        # counter threshold
        counter_line = DashedLine(start = self.center + self.tube_height/4*UP - self.tube_width/2*RIGHT, end = self.center - self.tube_height/4*UP - self.tube_width/2*RIGHT, color = self.tube_stroke_color)
        self.add(tube_background_polygon, tube_polygon)
        self.add(hallway_tunnel_up, hallway_tunnel_down, hallway_funnel_up, hallway_funnel_down, hallway_tunnel_up_left, hallway_tunnel_down_left, counter_line)

        # coordinate system: x in [0, tube_width], y in [-1, 1]
        self.hallway_ax = Axes(x_range = [-self.tube_width, 0, 1], y_range = [-1, 1, 1], x_length = self.tube_width, y_length = 2*self.tube_height, tips = False).set_color(BLACK).move_to(self.center)
        # self.add(self.hallway_ax)


    # returns path function for a given initial value y0
    def get_path(self, y0):
        def path_function(x):
            # left segment
            if x < -2.75:
                return 0.25 * y0
            # right segment
            elif x > -2.25:
                return y0
            # middle segment
            else:
                return 1.5*y0*x + (1+1.5*2.25)*y0
        return path_function
    

    # draws the path functions
    def draw_path(self, y0_list):
        plot_group = VGroup()
        for y0 in y0_list:
            path_plot_left = self.hallway_ax.plot(lambda x: 0.25 * y0, color = GRAY, stroke_width = 1, x_range = [-4.9, -2.75])
            path_plot_mid = self.hallway_ax.plot(lambda x: 1.5*y0*x + (1+1.5*2.25)*y0, color = GRAY, stroke_width = 1, x_range = [-2.75, -2.25])
            path_plot_right = self.hallway_ax.plot(lambda x: y0, color = GRAY, stroke_width = 1, x_range = [-2.25, 0])
            plot_group.add(path_plot_left, path_plot_mid, path_plot_right)
        return plot_group
    

    # calculate the velocity depending on the x coordinate
    def get_velocity(self, x):
        # left segment
        if x < -2.75:
            return -self.v0
        # right segment
        elif x > -2.25:
            return -self.v0
        # middle segment
        else:
            return -self.v0 * 0.25
        

    # calculate the x coordinate for a given time
    def update_pos(self, pos, y0, dt = 0.01):
        x, y, _ = pos
        new_x = x + self.get_velocity(x) * dt
        # set back the x coordinate and increase counter
        if new_x < -self.tube_width+0.1:
            new_x = 0
            self.count += 1
        new_y = self.get_path(y0)(new_x)
        return np.array([new_x, new_y, 0])
    

    # takes a position and returns the ax position
    def pos_to_ax(self, pos):
        return self.hallway_ax.c2p(*pos)
    

    # gets a dot
    def init_dot(self, y0):
        dot = Dot(radius = 0.05, color = GRAY)
        dot.y0 = y0
        dot.position = np.array([0, y0, 0])
        dot.move_to(self.pos_to_ax(dot.position))
        return dot



    def get_counter(self, n = 0):
        counter_bg = Rectangle(height = 1, width = self.tube_width/5*2, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = 0.95).move_to(self.center - 1.5/5*self.tube_width*RIGHT + 3*UP)
        counter_number = Text(f"{n}", color = BLACK).move_to(self.center - 1.5/5*self.tube_width*RIGHT + 3*UP)
        counter_bg.z_index = -1
        return VGroup(counter_bg, counter_number)

In [349]:
%%manim -qh --fps 60 $video_scene


class ndw_bernoulli_Scene(Scene):
    def construct(self):
        self.camera.background_color = background_color


        # bernoulli tube
        bernoulli_tube = BernoulliTube(tube_center = np.array([-3, -1, 0]), tube_stroke_color = BLACK)
        self.add(bernoulli_tube)

        bernoulli_tube_counter = bernoulli_tube.get_counter()
        self.add(bernoulli_tube_counter)

        # y0_list = [x/10 for x in range(-9, 10, 1)]
        y0_list = [
            0, 
            0.5,
            -0.6,
            0.1,
            -0.7,
            0.8,
            -0.1,
            0.3,
            0.9,
            -0.5,
            0.7,
            -0.4,
            -0.9,
            0.4,
            -0.3,
            0.2,
            0.6,
            -0.8,
            -0.2
            ]
        bernoulli_tube_streamlines = bernoulli_tube.draw_erf(y0_list)
        self.add(bernoulli_tube_streamlines)


        # bernoulli dots
        bernoulli_dot_list = [bernoulli_tube.init_dot(y0) for y0 in y0_list]
        for bernoulli_dot in bernoulli_dot_list:
            self.add(bernoulli_dot)


        def bernoulli_dot_updater(dot):
            y0 = dot.y0
            old_pos = dot.position
            new_pos = bernoulli_tube.update_pos(old_pos, y0)
            dot.position = new_pos
            dot.move_to(bernoulli_tube.pos_to_ax(new_pos))


        def bernoulli_counter_updater(counter):
            count = bernoulli_tube.count
            counter.become(bernoulli_tube.get_counter(count))


        # hallway with door
        door_hallway = DoorHallway(hallway_center = np.array([3, -1, 0]), tube_stroke_color = BLACK)
        self.add(door_hallway)

        door_hallway_counter = door_hallway.get_counter()
        self.add(door_hallway_counter)

        door_hallway_streamlines = door_hallway.draw_path(y0_list)
        self.add(door_hallway_streamlines)


        # hallway dots
        y0_hallway_list = [
            0, 
            0.5,
            -0.6,
            0.1,
            -0.7,
            0.8,
            -0.1,
            0.3,
            0.9,
            -0.5,
            0.7,
            -0.4,
            -0.9,
            0.4,
            -0.3,
            0.2,
            0.6,
            -0.8,
            -0.2
            ]
        hallway_dot_list = [door_hallway.init_dot(y0) for y0 in y0_hallway_list]
        for hallway_dot in hallway_dot_list:
            self.add(hallway_dot)


        def hallway_dot_updater(dot):
            y0 = dot.y0
            old_pos = dot.position
            new_pos = door_hallway.update_pos(old_pos, y0)
            dot.position = new_pos
            dot.move_to(door_hallway.pos_to_ax(new_pos))


        def hallway_counter_updater(counter):
            count = door_hallway.count
            counter.become(door_hallway.get_counter(count))


        time_tracker = ValueTracker(0)
        bernoulli_tube_counter.add_updater(bernoulli_counter_updater)
        door_hallway_counter.add_updater(hallway_counter_updater)
        for i in range(len(bernoulli_dot_list)):
            bernoulli_dot_list[i].add_updater(bernoulli_dot_updater)
            hallway_dot_list[i].add_updater(hallway_dot_updater)
            self.play(time_tracker.animate.set_value(2.75/19), rate_func = linear, run_time = 2.75/19)
        self.play(time_tracker.animate.set_value(10), rate_func = linear, run_time = 20)
        

Manim Community v0.18.1